In [ ]:
import os
import subprocess
import pandas as pd

os.makedirs('exploration', exist_ok=True)

malware_df = pd.read_csv('data/DikeDataset/labels/malware.csv')
malware_df['benign_or_malware'] = 'malware'

benign_df = pd.read_csv('data/DikeDataset/labels/benign.csv')
benign_df['benign_or_malware'] = 'benign'
benign_df = benign_df.astype(malware_df.dtypes.to_dict())

benign_df.dtypes

labels = pd.concat([benign_df, malware_df]).reset_index(drop=True)

In [ ]:
filepath = f'data/DikeDataset/files/{labels.iat[0, 12]}/{labels.iat[0, 1]}.exe'

subprocess.run(['python', 'main.py', '-o', f'{labels.iat[0, 1]}.png', filepath])

In [ ]:
with open('data/DikeDataset/files/benign/2a922ebe7edb08480baa1721ce1b5185fb5af7f64ec0f128d6a7a37711784815.exe')